### Web scraping by Python with Selenium <Beginner>

Reference: https://kaeroidata.medium.com/web-scraping-%E0%B8%94%E0%B9%89%E0%B8%A7%E0%B8%A2-python-%E0%B9%82%E0%B8%94%E0%B8%A2-selenium-beginner-ad6369624ccd

In [1]:
# Webpage for pactice IMDb : https://www.imdb.com/

In [2]:
# Install Selenium Module
# pip install selenium

1. Import Library

In [3]:
#Import modules
from selenium import webdriver
import pandas as pd
import os

2. Set directory(folder)

In [10]:
#Set directory
path = 'C:/web_selenium/' 
os.chdir(path) 

3. Simulate Chrome that use for control 

In [39]:
#Open webdriver 
driver = webdriver.Chrome(path+"chromedriver.exe")
driver.get('https://www.imdb.com/chart/top?sort=us,desc&mode=simple&page=1')

<ipython-input-39-cb30709e75a0>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path+"chromedriver.exe")


Movie Names

In [15]:
#Get location of movie names
movienames = driver.find_elements_by_class_name('titleColumn')

<ipython-input-15-a58d749057a0>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  movienames = driver.find_elements_by_class_name('titleColumn')


In [16]:
#Get movie names
Movienames=[name.text for name in movienames]

Rating

In [18]:
#Get ratings
ratings = driver.find_elements_by_class_name('ratingColumn.imdbRating')
Ratings=[rating.text for rating in ratings]

<ipython-input-18-34d834225f8b>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  ratings = driver.find_elements_by_class_name('ratingColumn.imdbRating')


Create DataFrame

In [19]:
#Convert to table (DataFrame) set Columns names: MovieNames and Ratings
DF_IMDb=pd.DataFrame({'MovieNames':Movienames , 'Ratings':Ratings})

Export to Excel file

In [20]:
#Export to Excel
DF_IMDb.to_excel('DF_IMDb_Data.xlsx')

### Keep Cast Name(Links)

In [21]:
#Get location of links
links=driver.find_elements_by_xpath("//td[@class='titleColumn']/a")

<ipython-input-21-cfac527161b9>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links=driver.find_elements_by_xpath("//td[@class='titleColumn']/a")


In [22]:
# จากนั้นจะใช้ method .get_attribute('href') ในการเก็บ link ออกมา ดังนี้
#Get links
Links = [link.get_attribute('href') for link in links]

In [23]:
# นำ link ที่ได้มาไปเก็บบน DataFrame
DF_IMDb['Links']=Links

In [24]:
DF_IMDb

,MovieNames,Ratings,Links
0,1. The Shawshank Redemption (1994),9.2,https://www.imdb.com/title/tt0111161/?pf_rd_m=...
1,2. The Godfather (1972),9.1,https://www.imdb.com/title/tt0068646/?pf_rd_m=...
2,3. The Godfather: Part II (1974),9.0,https://www.imdb.com/title/tt0071562/?pf_rd_m=...
3,4. The Dark Knight (2008),9.0,https://www.imdb.com/title/tt0468569/?pf_rd_m=...
4,5. 12 Angry Men (1957),8.9,https://www.imdb.com/title/tt0050083/?pf_rd_m=...
...,...,...,...
245,246. The Battle of Algiers (1966),8.0,https://www.imdb.com/title/tt0058946/?pf_rd_m=...
246,247. Nights of Cabiria (1957),8.0,https://www.imdb.com/title/tt0050783/?pf_rd_m=...
247,248. The Princess Bride (1987),8.0,https://www.imdb.com/title/tt0093779/?pf_rd_m=...
248,"249. Paris, Texas (1984)",8.0,https://www.imdb.com/title/tt0087884/?pf_rd_m=...


In [ ]:
'''หลังจากเราได้ link มาแล้ว เมื่อเข้าไปทีละลิ้งค์เราจะดึงอะไรออกมาได้บ้าง
1. ชื่อผู้กำกับ (Director)
2. รายชื่อนักแสดงและบทบาทที่แสดง (Cast)
3. โครงเรื่อง (Storyline) และประเภทหนัง (Genres)
4. ระยะเวลาของหนัง (Runtime)
5. วันที่หนังฉาย (Release Date)
6. รายได้รวมทั่วโลก ณ.ปัจจุบัน (Cumulative Worldwide Gross)'''

Testing pull 1 page before use loop and test code before pull all pages.

In [56]:
#Set List to keep data in each link
Director=[]
ActorNames =[]
CharacterNames =[]
Storyline=[]
Genres=[]

#Loop Links
for link in range(0,len(Links)):
    driver.get(Links[link])
    
    #Get Director
    director = driver.find_element_by_xpath("//div[@class='ipc-metadata-list-item__content-container']/a").text
    
    #Get casts (ActorNames,CharactorNames)
    casts = driver.find_elements_by_xpath("//table[@class='cast_list']/tbody/tr")
    Actornames =[cast.text.split('...')[0] for cast in casts if str(cast.text.find('...')).isnumeric()]
    Characternames =[cast.text.split('...')[1] for cast in casts if str(cast.text.find('...')).isnumeric()]
    
    #Get storyline
    storyline = driver.find_element_by_xpath("//div[@class='ipc-html-content ipc-html-content--base']").text
    genres = driver.find_elements_by_xpath("//div[@class='ipc-metadata-list-item__content-container']")[1].text
    genres2 =  genres[genres.find(" ")+1:]
    
    
    #Collect data in each website 
    Director.append(director)
    ActorNames.append(Actornames)
    CharacterNames.append(Characternames)
    Storyline.append(storyline)
    Genres.append(genres2)

<ipython-input-56-ce2594b49939>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  director = driver.find_element_by_xpath("//div[@class='ipc-metadata-list-item__content-container']/a").text
<ipython-input-56-ce2594b49939>:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  casts = driver.find_elements_by_xpath("//table[@class='cast_list']/tbody/tr")
<ipython-input-56-ce2594b49939>:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  storyline = driver.find_element_by_xpath("//div[@class='ipc-html-content ipc-html-content--base']").text
<ipython-input-56-ce2594b49939>:22: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  genres = driver.find_elements_by_xpath("//div[@class='ipc-metadata-list-item__content-container']")[1].text


In [57]:
#Collect all data to New DataFrame
DF_IMDb2=pd.DataFrame({'Director':Director,'ActorNames':ActorNames,'CharacterNames':CharacterNames,
                       'Storyline':Storyline,'Genres':Genres})

In [58]:
#Concatenate 2 DataFrame
DF_IMDb=pd.concat([DF_IMDb, DF_IMDb2], axis=1)

In [59]:
#Export to excel file
DF_IMDb.to_excel('DF_IMDb_Data.xlsx')